In [1]:
import sys
sys.path.insert(0, "/home/ubuntu/data/code/Modules/")
import skimage
import numpy as np
import pickle
import gc
from albumentations import Compose,HorizontalFlip, VerticalFlip, ToFloat
from sklearn.metrics import confusion_matrix
import cv2
import matplotlib.pyplot as plt
from keras import backend
from keras.models import Sequential
from keras.losses import binary_crossentropy
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers import Dense, BatchNormalization, Flatten, Dropout
from keras.optimizers import SGD,Adadelta
import DataGenerator
from albumentations import Compose,HorizontalFlip, VerticalFlip, ToFloat
from sklearn.model_selection import KFold
import random
import glob
import CNN_Module as cnn_module
import models
import DataGenerator_3d
import time
import warnings

Using TensorFlow backend.


In [29]:
import pickle
results=pickle.load(open('/home/ubuntu/data/code/chaining/3dchain_preds_final_ceph.p','rb'))
train_preds = results[0]
test_preds = results[1]
print(len(train_preds))
print(len(test_preds))

6585
1647


In [30]:
import Get_Simple_Fluctuations_Average_Intensity as simp

In [31]:
# finding length data
resistant_lengths = simp.getlengths('/home/ubuntu/data/cephdata/resistant/')
sus_lengths = simp.getlengths('/home/ubuntu/data/cephdata/susceptible/')
all_lengths = resistant_lengths+sus_lengths

In [32]:
# identifying labels
targets = simp.gettargets('/home/ubuntu/data/cephdata/resistant/','/home/ubuntu/data/cephdata/susceptible/')

In [33]:
# create new dictionary with paths and lengths
result_dict = {}
for path, length, _ in all_lengths:
    result_dict[path]=(length,targets[path])

In [7]:
import glue_code

In [34]:
#First find the cells in the train and test partitions
test_cells = sorted(glue_code.partition_to_keys(list(test_preds.keys())))
train_cells = sorted(glue_code.partition_to_keys(list(train_preds.keys())))

In [35]:
#go from list of cells, to list of lists of images from each cell
cell_ims_test = glue_code.split_preds_into_cells(test_preds,test_cells)
cell_ims_train = glue_code.split_preds_into_cells(train_preds,train_cells)

#get the av score for susceptible, and proportion of images classified as susceptible
cell_predictions_test = glue_code.get_cell_predictions(cell_ims_test,test_preds)
cell_predictions_train = glue_code.get_cell_predictions(cell_ims_train,train_preds)
#print(cell_predictions_test)

#glue together CNN predictions and av/lengths/labels
test_final = glue_code.glue_flucs_preds(cell_predictions_test,result_dict)
train_final = glue_code.glue_flucs_preds(cell_predictions_train,result_dict)

In [36]:
test_arr =(np.array(test_final)[:,1:].astype(float))
train_arr =  (np.array(train_final)[:,1:].astype(float))

np.random.shuffle(test_arr)
np.random.shuffle(train_arr)

In [37]:
# split off the labels which we are trying to predict, from the predictors
Xtrain = train_arr[:,:-1]
Ytrain = train_arr[:,-1]

Xtest = test_arr[:,:-1]
Ytest = test_arr[:,-1]

In [40]:
# train new algorithm
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier,BaggingClassifier,GradientBoostingClassifier
#clf = AdaBoostClassifier(n_estimators=3, random_state=0)
#clf = RandomForestClassifier(max_depth=1,n_estimators=1, random_state=0)
clf = GradientBoostingClassifier(n_estimators=10,max_depth=1)
clf.fit(Xtrain, Ytrain)
print(clf.score(Xtest, Ytest))

0.7638888888888888
